In [55]:
import pandas as pd 
##IMPORTING LIBRARY 

In [8]:
df = pd.read_csv(r"C:\Users\hp\Downloads\aml.csv")
df.head()

,transaction_id,customer_id,account_id,transaction_date,amount,country,transaction_type
0,T00001,C034,A592,31-12-2023,46712,Canada,Transfer
1,T00002,C041,A195,27-03-2023,199615,Canada,ACH
2,T00003,C040,A347,12-12-2023,151243,Canada,Transfer
3,T00004,C056,A876,05-06-2023,179260,Cayman Islands,Cash
4,T00005,C053,A113,07-01-2023,602,USA,Cash


In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    800 non-null    object
 1   customer_id       800 non-null    object
 2   account_id        800 non-null    object
 3   transaction_date  800 non-null    object
 4   amount            800 non-null    int64 
 5   country           800 non-null    object
 6   transaction_type  800 non-null    object
dtypes: int64(1), object(6)
memory usage: 43.9+ KB


In [14]:
df["transaction_date"].dtype


dtype('<M8[ns]')

In [15]:
df=df.sort_values(by=["account_id","transaction_date"])

In [56]:
##DEFINING HIGH RISK COUNTRIES
high_risk_countries = [
    "Iran", "North Korea", "Cayman Islands", "Panama"
]
df["high_risk_countries_flag"]=df["country"].isin(high_risk_countries)
df[["country", "high_risk_countries_flag"]].head(10)


,country,high_risk_countries_flag
278,Canada,False
348,India,False
742,USA,False
406,USA,False
490,UK,False
22,Canada,False
85,UK,False
766,Germany,False
535,North Korea,True
514,Germany,False


In [57]:
## SUMRFING (1ST RED FLAG)
df["structuring_flag"] = (
    (df["amount"] < 10000) &
    (df.groupby("account_id")["amount"]
       .transform(lambda x: x.rolling(5, min_periods=3).count() >= 3))
)
df[["account_id", "amount", "structuring_flag"]].head(10)



,account_id,amount,structuring_flag
278,A107,170271,False
348,A107,4339,False
742,A107,26975,False
406,A107,26251,False
490,A107,94643,False
22,A107,105906,False
85,A107,150598,False
766,A107,145635,False
535,A107,10236,False
514,A107,53255,False


In [36]:
##SPIKING ( ANOTHER RED FLAG)
avg_amount=df.groupby("account_id")["amount"].transform("mean")
df["spike_flag"]=df["amount"]>(3*avg_amount)
df[df["spike_flag"] == True][["account_id", "amount", "spike_flag"]].head(10)


,account_id,amount,spike_flag
693,A186,194854,True
251,A416,174034,True


In [38]:
df[[
    "transaction_id",
    "account_id",
    "amount",
    "country",
    "structuring_flag",
    "high_risk_countries_flag",
    "spike_flag"
]].head(10)


,transaction_id,account_id,amount,country,structuring_flag,high_risk_countries_flag,spike_flag
278,T00279,A107,170271,Canada,False,False,False
348,T00349,A107,4339,India,False,False,False
742,T00743,A107,26975,USA,False,False,False
406,T00407,A107,26251,USA,False,False,False
490,T00491,A107,94643,UK,False,False,False
22,T00023,A107,105906,Canada,False,False,False
85,T00086,A107,150598,UK,False,False,False
766,T00767,A107,145635,Germany,False,False,False
535,T00536,A107,10236,North Korea,False,True,False
514,T00515,A107,53255,Germany,False,False,False


In [39]:
df["account_id"].unique()


array(['A107', 'A113', 'A122', 'A123', 'A147', 'A148', 'A186', 'A187',
       'A195', 'A214', 'A224', 'A234', 'A269', 'A278', 'A336', 'A347',
       'A348', 'A405', 'A416', 'A456', 'A462', 'A463', 'A470', 'A501',
       'A503', 'A547', 'A576', 'A588', 'A592', 'A600', 'A606', 'A635',
       'A643', 'A658', 'A665', 'A667', 'A679', 'A684', 'A691', 'A706',
       'A737', 'A750', 'A752', 'A761', 'A764', 'A787', 'A820', 'A859',
       'A875', 'A876', 'A896', 'A908', 'A909', 'A912', 'A921', 'A962',
       'A974', 'A977', 'A994', 'A998'], dtype=object)

In [43]:
alerts_df = df[
    (df["structuring_flag"] == True) |
    (df["high_risk_countries_flag"] == True) |
    (df["spike_flag"] == True)
]

alerts_df[[
    "transaction_id",
    "account_id",
    "amount",
    "country",
    "structuring_flag",
    "high_risk_countries_flag",
    "spike_flag"
]].head(10)



,transaction_id,account_id,amount,country,structuring_flag,high_risk_countries_flag,spike_flag
535,T00536,A107,10236,North Korea,False,True,False
192,T00193,A113,83854,Panama,False,True,False
733,T00734,A113,73350,Panama,False,True,False
127,T00128,A113,4418,Germany,True,False,False
132,T00133,A122,24785,Panama,False,True,False
385,T00386,A122,9227,Canada,True,False,False
583,T00584,A122,8941,Iran,True,True,False
153,T00154,A122,3622,USA,True,False,False
767,T00768,A122,8656,Cayman Islands,True,True,False
502,T00503,A123,94737,Cayman Islands,False,True,False


In [45]:
##CREATION OF RISK SCORE FOR EACH TRANSACTION 
df["risk_score"]= (
    df["structuring_flag"].astype(int)*40+
    df["high_risk_countries_flag"].astype(int)*30+
    df["spike_flag"].astype(int)*30)
    
    

In [47]:
df[[
    "transaction_id",
    "account_id",
    "amount",
    "structuring_flag",
    "high_risk_countries_flag",
    "spike_flag",
    "risk_score"
]].tail(10)


,transaction_id,account_id,amount,structuring_flag,high_risk_countries_flag,spike_flag,risk_score
299,T00300,A998,187902,False,True,False,30
640,T00641,A998,88022,False,False,False,0
612,T00613,A998,193794,False,False,False,0
228,T00229,A998,9882,True,False,False,40
565,T00566,A998,152896,False,False,False,0
305,T00306,A998,113011,False,False,False,0
37,T00038,A998,113802,False,False,False,0
712,T00713,A998,197497,False,False,False,0
602,T00603,A998,107180,False,False,False,0
329,T00330,A998,133427,False,False,False,0


In [49]:
## ASSIGNING RISK CATEGORY HIGH/MEDIUM/LOW
def assign_risk_category(score):
    if score >= 70:
        return "High"
    elif score >= 40:
        return "Medium"
    else:
        return "Low"

df["risk_category"] = df["risk_score"].apply(assign_risk_category)
df[[
    "transaction_id",
    "account_id",
    "risk_score",
    "risk_category"
]].head(10)


,transaction_id,account_id,risk_score,risk_category
278,T00279,A107,0,Low
348,T00349,A107,0,Low
742,T00743,A107,0,Low
406,T00407,A107,0,Low
490,T00491,A107,0,Low
22,T00023,A107,0,Low
85,T00086,A107,0,Low
766,T00767,A107,0,Low
535,T00536,A107,30,Low
514,T00515,A107,0,Low


In [50]:
alerts_final = df[df["risk_category"] == "High"]


In [53]:
##CREATION OF ACCOUNT SUMMARY AND DEFINING THE RISK CATEGORY OF ACCOUNT
account_risk_summary = df.groupby("account_id").agg(
    total_transactions=("transaction_id", "count"),
    total_alerts=("risk_category", lambda x: (x == "High").sum()),
    max_risk_score=("risk_score", "max"),
    avg_transaction_amount=("amount", "mean")
).reset_index()
def account_risk_level(row):
    if row["total_alerts"] >= 2 or row["max_risk_score"] >= 70:
        return "High"
    elif row["total_alerts"] == 1:
        return "Medium"
    else:
        return "Low"

account_risk_summary["account_risk_level"] = account_risk_summary.apply(
    account_risk_level, axis=1
)

account_risk_summary.head(10)


,account_id,total_transactions,total_alerts,max_risk_score,avg_transaction_amount,account_risk_level
0,A107,13,0,30,78108.692308,Low
1,A113,18,0,40,98194.111111,Low
2,A122,18,2,70,81733.333333,High
3,A123,10,0,30,101968.700000,Low
4,A147,9,0,40,87555.888889,Low
5,A148,10,0,40,116741.200000,Low
6,A186,16,0,40,64625.000000,Low
7,A187,18,0,40,67182.333333,Low
8,A195,20,0,40,91343.200000,Low
9,A214,13,0,40,82419.615385,Low


In [54]:
account_risk_summary.to_csv("account_risk_summary.csv", index=False)
